# VGG-16 Train All Layer with Data Augmentation 

In [1]:
# All the layers of VGG-16 trained and last Fully connected layer is changed accordingly
# 17 participants data 
# With Data Augmentation


In [2]:
import numpy as np
from keras.models import Sequential
from keras.layers.convolutional import Conv3D
from keras.layers.convolutional_recurrent import ConvLSTM2D
from keras.layers.normalization import BatchNormalization
from keras.callbacks import TensorBoard, EarlyStopping, ModelCheckpoint
from keras.layers import Dense, Activation
from keras.layers import Input
from keras.utils import multi_gpu_model
from keras import optimizers
from keras.preprocessing.image import ImageDataGenerator
from keras.layers import Input
from keras.layers import Flatten,Dropout
from keras.layers import Conv2D, MaxPooling2D
from keras.utils import to_categorical
from keras.preprocessing import image 
from keras.models import Model
from keras.applications.vgg16 import VGG16, preprocess_input



Using TensorFlow backend.


###### Loading Data 

In [3]:
x_train = np.load('/cai_lab/thomas/DWD/Numpy_RGB/train/x_data_all.npy')     # Data Path
y_train = np.load('/cai_lab/thomas/DWD/Numpy_RGB/train/y_data_all.npy')

x_valid = np.load('/cai_lab/thomas/DWD/Numpy_RGB/valid/x_data_all.npy')
y_valid = np.load('/cai_lab/thomas/DWD/Numpy_RGB/valid/y_data_all.npy')

print('x_train.shape:',x_train.shape,'y_train.shape:',y_train.shape)
print('x_valid.shape:',x_valid.shape,'y_valid.shape:',y_valid.shape)

x_train.shape: (17194, 224, 224, 3) y_train.shape: (17194, 9)
x_valid.shape: (2558, 224, 224, 3) y_valid.shape: (2558, 9)


In [4]:
batch_size = 64
resize = 224
n_class = 9

### CNN Architecture 

In [5]:
from keras.layers import Input
vgg16_input = Input(shape = (224, 224, 3), name = 'Image_input')


##  VGG model

from keras.applications.vgg16 import VGG16, preprocess_input

#Getting the CNN layer of the VGG network trained on ImageNet
model_vgg16_conv = VGG16(weights='imagenet', include_top=False, input_tensor = vgg16_input)
model_vgg16_conv.summary()

W0419 04:20:38.676025 140131208906496 deprecation_wrapper.py:119] From /home/cai/.local/lib/python3.5/site-packages/keras/backend/tensorflow_backend.py:4070: The name tf.nn.max_pool is deprecated. Please use tf.nn.max_pool2d instead.



Model: "vgg16"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
Image_input (InputLayer)     (None, 224, 224, 3)       0         
_________________________________________________________________
block1_conv1 (Conv2D)        (None, 224, 224, 64)      1792      
_________________________________________________________________
block1_conv2 (Conv2D)        (None, 224, 224, 64)      36928     
_________________________________________________________________
block1_pool (MaxPooling2D)   (None, 112, 112, 64)      0         
_________________________________________________________________
block2_conv1 (Conv2D)        (None, 112, 112, 128)     73856     
_________________________________________________________________
block2_conv2 (Conv2D)        (None, 112, 112, 128)     147584    
_________________________________________________________________
block2_pool (MaxPooling2D)   (None, 56, 56, 128)       0     

#### Modified Fully connected Layer 

In [6]:
output_vgg16_conv = model_vgg16_conv(vgg16_input)

#Add the fully-connected layers 

x = Flatten(name='flatten')(output_vgg16_conv)
x = Dense(4096, activation='relu', name='fc1')(x)
x = Dense(4096, activation='relu', name='fc2')(x)
x = Dense(n_class, activation='softmax', name='predictions')(x)

vgg16_pretrained = Model(input = vgg16_input, output = x)
vgg16_pretrained.summary()



Model: "model_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
Image_input (InputLayer)     (None, 224, 224, 3)       0         
_________________________________________________________________
vgg16 (Model)                (None, 7, 7, 512)         14714688  
_________________________________________________________________
flatten (Flatten)            (None, 25088)             0         
_________________________________________________________________
fc1 (Dense)                  (None, 4096)              102764544 
_________________________________________________________________
fc2 (Dense)                  (None, 4096)              16781312  
_________________________________________________________________
predictions (Dense)          (None, 9)                 36873     
Total params: 134,297,417
Trainable params: 134,297,417
Non-trainable params: 0
_____________________________________________

### Model Saving and Compling 

In [7]:

save_name = 'VGG-16_TrainallLayer_withDataAug__RGB_200418'
save_path = './checkpoints/' + save_name

if os.path.exists(save_path)==False:
    os.makedirs(save_path)

checkpoint = ModelCheckpoint(save_path +'/checkpoint-{epoch:04d}.h5', verbose=1, monitor='val_loss', save_best_only=False, mode='auto', period=1)
tensorboard = TensorBoard(log_dir='./graph/'+ save_name, histogram_freq=0, write_graph=True, write_images=True)
callback_list = [checkpoint, tensorboard]

vgg16_pretrained = multi_gpu_model(vgg16_pretrained, gpus=2)

aug = ImageDataGenerator(
        rotation_range=20,
        zoom_range=0.15,
        width_shift_range=0.2,
        height_shift_range=0.2,
        shear_range=0.15,
        horizontal_flip=True,
        fill_mode="nearest")


# Compile CNN model
sgd = optimizers.SGD(lr = 0.001)
vgg16_pretrained.compile(loss='categorical_crossentropy', optimizer = sgd, metrics=['accuracy'])

    
vgg16_pretrained.fit_generator(aug.flow(x_train, y_train, batch_size=batch_size),
                    validation_data=(x_valid, y_valid),
                    steps_per_epoch = len(x_train) // batch_size,        
                    epochs=100,
                    verbose=1,
                    callbacks=callback_list)


vgg16_pretrained.save(save_path + '/epoch-finish.h5')
vgg16_pretrained.save_weights(save_path +'/epoch-finish_weights.h5')

W0419 04:20:48.087717 140131208906496 deprecation.py:323] From /home/cai/.local/lib/python3.5/site-packages/tensorflow/python/ops/math_grad.py:1250: add_dispatch_support.<locals>.wrapper (from tensorflow.python.ops.array_ops) is deprecated and will be removed in a future version.
Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where
W0419 04:20:52.165278 140131208906496 deprecation_wrapper.py:119] From /home/cai/.local/lib/python3.5/site-packages/keras/backend/tensorflow_backend.py:422: The name tf.global_variables is deprecated. Please use tf.compat.v1.global_variables instead.

W0419 04:20:52.173036 140131208906496 deprecation_wrapper.py:119] From /home/cai/.local/lib/python3.5/site-packages/keras/callbacks/tensorboard_v1.py:200: The name tf.summary.merge_all is deprecated. Please use tf.compat.v1.summary.merge_all instead.

W0419 04:20:52.173726 140131208906496 deprecation_wrapper.py:119] From /home/cai/.local/lib/python3.5/site-packages/keras

Epoch 1/100
268/268 [==============================] - 220s 821ms/step - loss: 1.9511 - accuracy: 0.3189 - val_loss: 1.3648 - val_accuracy: 0.4961

Epoch 00001: saving model to ./checkpoints/VGG-16_TrainallLayer_withDataAug__RGB_200418/checkpoint-0001.h5


W0419 04:26:04.168115 140131208906496 deprecation_wrapper.py:119] From /home/cai/.local/lib/python3.5/site-packages/keras/callbacks/tensorboard_v1.py:343: The name tf.Summary is deprecated. Please use tf.compat.v1.Summary instead.



Epoch 2/100
268/268 [==============================] - 218s 814ms/step - loss: 0.9478 - accuracy: 0.6773 - val_loss: 1.1191 - val_accuracy: 0.5887

Epoch 00002: saving model to ./checkpoints/VGG-16_TrainallLayer_withDataAug__RGB_200418/checkpoint-0002.h5
Epoch 3/100
268/268 [==============================] - 217s 808ms/step - loss: 0.4727 - accuracy: 0.8505 - val_loss: 1.1168 - val_accuracy: 0.6982

Epoch 00003: saving model to ./checkpoints/VGG-16_TrainallLayer_withDataAug__RGB_200418/checkpoint-0003.h5
Epoch 4/100
268/268 [==============================] - 215s 802ms/step - loss: 0.3117 - accuracy: 0.8994 - val_loss: 1.1131 - val_accuracy: 0.6931

Epoch 00004: saving model to ./checkpoints/VGG-16_TrainallLayer_withDataAug__RGB_200418/checkpoint-0004.h5
Epoch 5/100
268/268 [==============================] - 215s 804ms/step - loss: 0.2258 - accuracy: 0.9295 - val_loss: 0.9165 - val_accuracy: 0.7295

Epoch 00005: saving model to ./checkpoints/VGG-16_TrainallLayer_withDataAug__RGB_200418

268/268 [==============================] - 207s 772ms/step - loss: 0.0252 - accuracy: 0.9922 - val_loss: 1.6405 - val_accuracy: 0.6845

Epoch 00034: saving model to ./checkpoints/VGG-16_TrainallLayer_withDataAug__RGB_200418/checkpoint-0034.h5
Epoch 35/100
268/268 [==============================] - 207s 773ms/step - loss: 0.0221 - accuracy: 0.9929 - val_loss: 1.3116 - val_accuracy: 0.7158

Epoch 00035: saving model to ./checkpoints/VGG-16_TrainallLayer_withDataAug__RGB_200418/checkpoint-0035.h5
Epoch 36/100
268/268 [==============================] - 208s 777ms/step - loss: 0.0237 - accuracy: 0.9922 - val_loss: 2.0212 - val_accuracy: 0.6298

Epoch 00036: saving model to ./checkpoints/VGG-16_TrainallLayer_withDataAug__RGB_200418/checkpoint-0036.h5
Epoch 37/100
268/268 [==============================] - 208s 775ms/step - loss: 0.0199 - accuracy: 0.9937 - val_loss: 1.4093 - val_accuracy: 0.6962

Epoch 00037: saving model to ./checkpoints/VGG-16_TrainallLayer_withDataAug__RGB_200418/checkpoi

268/268 [==============================] - 208s 778ms/step - loss: 0.0043 - accuracy: 0.9989 - val_loss: 2.0088 - val_accuracy: 0.6720

Epoch 00098: saving model to ./checkpoints/VGG-16_TrainallLayer_withDataAug__RGB_200418/checkpoint-0098.h5
Epoch 99/100
268/268 [==============================] - 207s 771ms/step - loss: 0.0056 - accuracy: 0.9980 - val_loss: 1.6032 - val_accuracy: 0.6818

Epoch 00099: saving model to ./checkpoints/VGG-16_TrainallLayer_withDataAug__RGB_200418/checkpoint-0099.h5
Epoch 100/100
268/268 [==============================] - 208s 775ms/step - loss: 0.0053 - accuracy: 0.9981 - val_loss: 1.8118 - val_accuracy: 0.6603

Epoch 00100: saving model to ./checkpoints/VGG-16_TrainallLayer_withDataAug__RGB_200418/checkpoint-0100.h5
